In [ ]:
from sensor_core import data_manager
from sensor_core import plot_manager
from sensor_core import strg_manager
from sensor_core import ser_manager
from sensor_core import mem_manager
from sensor_core import utils
import multiprocessing

In [ ]:
ser_manager.find_serial()

In [ ]:
# User Defined Parameters
channel_key = ["red", "infrared", "violet"]  # Names of channels
commport = '/dev/ttyACM1'                    # Name of comm port
baudrate = 115200                            # bits/second
num_points = 1000                            # Number of points to plot, and to save per cycle
window_size = 1                              # Number of points to acquire before updating plot

In [ ]:
# Setup Serial Manager
sm = ser_manager.SerialManager(commport=commport, baudrate=baudrate)
sm.setup_serial()

In [ ]:
# Setup Memory Manager
mutex = mem_manager.create_mutex()
shm, data_shared = mem_manager.create_shared_block(channel_key=channel_key, num_points=num_points)

In [ ]:
# Initialize Static Arguments Dictionary
static_args_dict = {
        "channel_key": channel_key,
        "commport": commport,
        "baudrate": baudrate,
        "mutex": mutex,
        "shm_name": shm.name,
        "shape": data_shared.shape,
        "dtype": data_shared.dtype,
        "EOL": None,
        "num_points": num_points
    }

In [ ]:
# Initialize Online Data Manager (without dynamic kwarg updates)
odm = data_manager.OnlineDataManager(static_args_dict=static_args_dict)

In [ ]:
# Initialize Plot Manager
pm = plot_manager.PlotManager(static_args_dict=static_args_dict)

In [ ]:
# Acquire and Update Data in Separate Process, Instantiate plot
p = multiprocessing.Process(name='update',
                            target=odm.online_update_data)
plot = pm.initialize_plot()

In [ ]:
# Start process, add callback function to plot to update plotting buffer data
p.start()
plot.add_animations(pm.online_grid_plot_data)

In [ ]:
# Show Plot
plot.show()